## Setup: Install Dependencies
Run this once to load project libs (requests, pandas, etc.).

In [1]:
# Install from requirements.txt (run once)
%pip install -r ../requirements.txt
print('✅ Deps installed—restart kernel if needed (Kernel > Restart).')

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
✅ Deps installed—restart kernel if needed (Kernel > Restart).



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\kumar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# AIOps Anomaly Detection: Interactive Demo

**Overview**: Offline simulation of automated anomaly detection for IT ops (e.g., Azure VM CPU metrics). 
- Learns 'normal' patterns from time-series data.
- Flags deviations without manual thresholds.
- Visualizes actual vs. expected for proactive downtime prevention.

**Tune & Re-run**: Change `anomaly_prob` or `num_points` in Cell 3, then re-execute.

Future: Replace mocks with Azure Monitor fetches for real data.

In [2]:
# Imports (path to root for src/)
%matplotlib inline 
import matplotlib.pyplot as plt
plt.ioff()  # Disable interactive mode (stops empty windows)
import sys
import random
sys.path.append('..')
from src.detect_anomalies import create_payload_with_anomalies
from src.visualize_anomalies import plot_detection
from datetime import datetime

In [3]:
# Step 1: Generate synthetic time-series data
# Simulates metrics like CPU % with patterns + random anomalies (spikes/drops)
random.seed(42)  # Reproducible for demos
payload = create_payload_with_anomalies(
    num_points=50,  # e.g., 50 mins of data
    anomaly_prob=0.15,  # 15% deviation chance (tune here!)
    base_value=75  # Normal baseline
)
print(f"Generated {len(payload['series'])} points with ~{int(0.15*50)} expected anomalies.")
print("Sample data:", payload['series'][:3])  # Peek at first points

Generated 50 points with ~7 expected anomalies.
Sample data: [{'timestamp': '2025-10-09T13:39:29.942630', 'value': 101.89485435196121}, {'timestamp': '2025-10-09T13:38:29.942630', 'value': 77.76699487422911}, {'timestamp': '2025-10-09T13:37:29.942630', 'value': 72.86938832629416}]


In [4]:
# Step 2: Mock Detection (AIOps 'learns' patterns & flags deviations)
# Real: Call Azure Anomaly Detector API with env vars
result = {
    'isAnomaly': True,  # Overall flag
    'expectedValues': [75] * len(payload['series']),  # Mock learned baseline
    'anomalyStatus': [1 if random.random() < 0.15 else 0 for _ in payload['series']]  # Per-point flags
}
anomaly_count = sum(result['anomalyStatus'])
print(f"Mock detection: {anomaly_count} anomalies flagged! 🚨\n(Proactive alert: Auto-scale resources?)")

Mock detection: 9 anomalies flagged! 🚨
(Proactive alert: Auto-scale resources?)


In [5]:
# Step 3: Visualize (actual vs. expected, red dots on deviations)
import matplotlib.pyplot as plt  # For post-plot saves

# Debug: Print payload to confirm data exists
print("Debug: Payload series length:", len(payload['series']))
print("Debug: Sample values:", [p['value'] for p in payload['series'][:5]])

plot_detection(payload, result, save_path='anomaly_demo_nb.png', show_plot=False)  # No plt.show() = no pop-up
print("Plot rendered—check inline/PNG for spikes & reds!")

# Tweak: Auto-save timestamped high-res PNG to repo (for commits)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_name = f"anomaly_demo_{timestamp}.png"
plt.savefig(save_name, dpi=150, bbox_inches='tight', facecolor='white')  # Saves open figure
print(f"📸 Timestamped plot saved: {save_name} (check file size >50KB for content)")

Debug: Payload series length: 50
Debug: Sample values: [101.89485435196121, 77.76699487422911, 72.86938832629416, 73.2979721943807, 65.55239725882886]
📊 Plot saved to anomaly_demo_nb.png
Plot rendered—check inline/PNG for spikes & reds!
📸 Timestamped plot saved: anomaly_demo_20251009_133930.png (check file size >50KB for content)


In [6]:
# Step 2.5: Azure Data Fetch Stub (Offline Mock for Now)
# Real: Use azure-monitor-query SDK (add to requirements.txt when live)
def fetch_metrics_mock(workspace_id="mock-ws-123", num_points=50):
    """
    Mocks Log Analytics query: e.g., 'Perf | where ObjectName == "Processor" | summarize avg(CounterValue) by bin(TimeGenerated, 1m)'
    Returns payload-ready series.
    """
    import random
    from datetime import timedelta
    base_ts = datetime.now()
    series = []
    for i in range(num_points):
        ts = (base_ts - timedelta(minutes=i)).isoformat()
        value = 75 + random.uniform(-10, 10) + (i % 5)  # Simulated CPU avg
        if random.random() < 0.1:  # 10% anomaly injection
            value += random.uniform(20, 50)  # Spike
        series.append({"timestamp": ts, "value": value})
    return {"series": series, "granularity": "PT1M"}

# Toggle for 'real' mode (swap mocks when Azure up)
use_azure_stub = True  # Set False for synthetic only
if use_azure_stub:
    payload = fetch_metrics_mock(num_points=50)  # Overrides previous payload
    print("🔄 Using Azure Monitor stub—real query swaps in here!")
else:
    # Fall back to synthetic
    payload = create_payload_with_anomalies(50, anomaly_prob=0.15)
    print("🔄 Using synthetic data.")

print(f"Payload ready: {len(payload['series'])} points from 'Azure'.")

🔄 Using Azure Monitor stub—real query swaps in here!
Payload ready: 50 points from 'Azure'.


In [7]:
# Step 4: Mock Alerting (Proactive Response on Anomalies)
# Real: Post to Azure Action Group webhook or email
alert_threshold = 3  # e.g., alert if >3 anomalies in window
if anomaly_count > alert_threshold:
    alert_msg = f"""
🚨 AIOps Alert: High Deviation Detected!
- Metrics: CPU % (simulated)
- Anomalies: {anomaly_count} in last {len(payload['series'])} mins
- Top Deviation: Max spike {max([p['value'] for p in payload['series']])}% at ~{payload['series'][0]['timestamp']}
- Action: Auto-scale resources or investigate logs.
- Severity: Medium (tune for multi-cloud)
"""
    print(alert_msg)
    
    # Mock 'send' (e.g., to email/Slack)
    print("📧 Mock: Alert sent to Ops team via Azure Alerts/Teams.")
else:
    print("✅ No alerts triggered—all within normal patterns.")


🚨 AIOps Alert: High Deviation Detected!
- Metrics: CPU % (simulated)
- Anomalies: 9 in last 50 mins
- Top Deviation: Max spike 125.72525824591895% at ~2025-10-09T13:39:31.077703
- Action: Auto-scale resources or investigate logs.
- Severity: Medium (tune for multi-cloud)

📧 Mock: Alert sent to Ops team via Azure Alerts/Teams.


In [8]:
# Step 4.5: AWS CloudWatch Mock (Multi-Cloud Tease)
# Real: Use boto3.client('cloudwatch').get_metric_statistics() for EC2 CPU
def fetch_cloudwatch_mock(instance_id="i-mock123", num_points=50):
    """
    Mocks CloudWatch query: e.g., CPUUtilization for EC2 instance over 50 mins.
    Returns payload-ready series.
    """
    import random
    from datetime import timedelta
    base_ts = datetime.now()
    series = []
    for i in range(num_points):
        ts = (base_ts - timedelta(minutes=i)).isoformat()
        value = 70 + random.uniform(-8, 12) + (i % 8)  # Simulated CPU with trend
        if random.random() < 0.12:  # 12% anomaly
            value += random.uniform(15, 45)  # Burst
        series.append({"timestamp": ts, "value": value})
    return {"series": series, "granularity": "PT1M"}

# Toggle for AWS mode
use_aws_mock = True  # Set False for Azure/synthetic
if use_aws_mock:
    payload = fetch_cloudwatch_mock(num_points=50)
    print("☁️ Using AWS CloudWatch mock—real boto3 swaps in here!")
else:
    # Fall back
    payload = fetch_metrics_mock(num_points=50)  # From Azure stub
    print("☁️ Using Azure stub.")

print(f"Payload ready: {len(payload['series'])} points from 'AWS'.")

# Re-run detection & viz for comparison (quick re-execute)
result = {
    'isAnomaly': True,
    'expectedValues': [70] * len(payload['series']),  # AWS baseline tweak
    'anomalyStatus': [1 if random.random() < 0.12 else 0 for _ in payload['series']]
}
anomaly_count = sum(result['anomalyStatus'])
print(f"AWS Mock detection: {anomaly_count} anomalies flagged! 🚨")

plot_detection(payload, result, save_path='aws_anomaly_demo.png', show_plot=True)
print("AWS plot rendered—compare spikes to Azure!")

☁️ Using AWS CloudWatch mock—real boto3 swaps in here!
Payload ready: 50 points from 'AWS'.
AWS Mock detection: 3 anomalies flagged! 🚨
📊 Plot saved to aws_anomaly_demo.png
AWS plot rendered—compare spikes to Azure!


In [9]:
# Step 4.75: GCP Operations Suite Mock (Full Multi-Cloud Trifecta)
# Real: Use google-cloud-monitoring SDK for VM instance metrics
def fetch_gcp_mock(project_id="my-project-123", num_points=50):
    """
    Mocks GCP Operations Suite query: e.g., CPU utilization for Compute Engine VM over 50 mins.
    Returns payload-ready series.
    """
    import random
    from datetime import timedelta
    base_ts = datetime.now()
    series = []
    for i in range(num_points):
        ts = (base_ts - timedelta(minutes=i)).isoformat()
        value = 72 + random.uniform(-9, 11) + (i % 6)  # Simulated CPU with seasonality
        if random.random() < 0.11:  # 11% anomaly
            value += random.uniform(18, 42)  # Surge
        series.append({"timestamp": ts, "value": value})
    return {"series": series, "granularity": "PT1M"}

# Toggle for GCP mode
use_gcp_mock = True  # Set False for Azure/AWS/synthetic
if use_gcp_mock:
    payload = fetch_gcp_mock(num_points=50)
    print("🌐 Using GCP Operations Suite mock—real SDK swaps in here!")
else:
    # Fall back to AWS
    payload = fetch_cloudwatch_mock(num_points=50)
    print("🌐 Using AWS mock.")

print(f"Payload ready: {len(payload['series'])} points from 'GCP'.")

# Re-run detection & viz for comparison
result = {
    'isAnomaly': True,
    'expectedValues': [72] * len(payload['series']),  # GCP baseline tweak
    'anomalyStatus': [1 if random.random() < 0.11 else 0 for _ in payload['series']]
}
anomaly_count = sum(result['anomalyStatus'])
print(f"GCP Mock detection: {anomaly_count} anomalies flagged! 🚨")

plot_detection(payload, result, save_path='gcp_anomaly_demo.png', show_plot=True)
print("GCP plot rendered—compare to Azure/AWS!")

🌐 Using GCP Operations Suite mock—real SDK swaps in here!
Payload ready: 50 points from 'GCP'.
GCP Mock detection: 5 anomalies flagged! 🚨
📊 Plot saved to gcp_anomaly_demo.png
GCP plot rendered—compare to Azure/AWS!


| **GCP** | Operations Suite | Alerting policies to Pub/Sub. |

## Multi-Cloud Extension + Alerting

| Cloud | Data Source | Alert Integration |
|-------|-------------|-------------------|
| **Azure** | Monitor/Log Analytics | Action Groups + Logic Apps for emails/SMS. |
| **AWS** | CloudWatch | SNS notifications on alarms. |
| **GCP** | Operations Suite | Alerting policies to Pub/Sub. |

**Next Steps**: 
- Wire mocks to real webhooks.
- Test with simulated downtime (e.g., anomaly_prob=0.5).